# Images and Physical Space

## Overview

In [ ]:
# Let's import matplotlib so we can view images
from matplotlib import pyplot as plt
%matplotlib inline

# Also import NumPy for working with arrays
import numpy as np

### The primary data type is the Image

In [ ]:
import SimpleITK as sitk

image = sitk.Image()
print(image)

### Use `ReadImage()` and `WriteImage()` for IO

In [ ]:
image = sitk.ReadImage('Data/qdna1.mha')

sitk.WriteImage(image, 'qdna1.tif')
!ls -l qdna1.tif

### Spatial computations are always done in physical space

This is very important to **avoid errors** when performing registration.

![Physical space](Data/PhysicalSpace.png)

### *Origin*, *Spacing*, and *Direction* cosines define conversion from index to physical space

In [ ]:
print('Origin', image.GetOrigin())
print('Spacing', image.GetSpacing())
print('Direction', image.GetDirection())

index = (0, 3)
point = image.TransformIndexToPhysicalPoint(index)
print('\nindex', index, 'corresponds to point', point)

In [ ]:
image.SetSpacing((3.0, 5.0))
print('Spacing', image.GetSpacing())

point = image.TransformIndexToPhysicalPoint(index)
print('\nindex', index, 'corresponds to point', point)

## Tutorial

### Conversion between NumPy and SimpleITK

SimpleITK and NumPy indexing access is in opposite order! 

SimpleITK: image[x,y,z]<br>
NumPy: image_numpy_array[z,y,x]

#### From SimpleITK to NumPy

In [ ]:
nda = sitk.GetArrayFromImage(image)
print(image.GetSize())
print(nda.shape)

#### From NumPy to SimpleITK

Remember to to set the image's origin, spacing, and possibly direction cosine matrix. The default values may not match the physical dimensions of your image.

In [ ]:
nda = np.zeros((10,20,3))

# if this is supposed to be a 3D gray scale image [x=3, y=20, z=10]
nda_as_image = sitk.GetImageFromArray(nda)
print(nda_as_image.GetSize())

# if this is supposed to be a 2D color image [x=20,y=10]
nda_as_image = sitk.GetImageFromArray(nda, isVector=True)
print(nda_as_image.GetSize())

### Pixel Types

The pixel type is represented as an enumerated type. The following is a table of the enumerated list.

<table>
  <tr><td>sitkUInt8</td><td>Unsigned 8 bit integer</td></tr>
  <tr><td>sitkInt8</td><td>Signed 8 bit integer</td></tr>
  <tr><td>sitkUInt16</td><td>Unsigned 16 bit integer</td></tr>
  <tr><td>sitkInt16</td><td>Signed 16 bit integer</td></tr>
  <tr><td>sitkUInt32</td><td>Unsigned 32 bit integer</td></tr>
  <tr><td>sitkInt32</td><td>Signed 32 bit integer</td></tr>
  <tr><td>sitkUInt64</td><td>Unsigned 64 bit integer</td></tr>
  <tr><td>sitkInt64</td><td>Signed 64 bit integer</td></tr>
  <tr><td>sitkFloat32</td><td>32 bit float</td></tr>
  <tr><td>sitkFloat64</td><td>64 bit float</td></tr>
  <tr><td>sitkComplexFloat32</td><td>complex number of 32 bit float</td></tr>
  <tr><td>sitkComplexFloat64</td><td>complex number of 64 bit float</td></tr>
  <tr><td>sitkVectorUInt8</td><td>Multi-component of unsigned 8 bit integer</td></tr>
  <tr><td>sitkVectorInt8</td><td>Multi-component of signed 8 bit integer</td></tr>
  <tr><td>sitkVectorUInt16</td><td>Multi-component of unsigned 16 bit integer</td></tr>
  <tr><td>sitkVectorInt16</td><td>Multi-component of signed 16 bit integer</td></tr>
  <tr><td>sitkVectorUInt32</td><td>Multi-component of unsigned 32 bit integer</td></tr>
  <tr><td>sitkVectorInt32</td><td>Multi-component of signed 32 bit integer</td></tr>
  <tr><td>sitkVectorUInt64</td><td>Multi-component of unsigned 64 bit integer</td></tr>
  <tr><td>sitkVectorInt64</td><td>Multi-component of signed 64 bit integer</td></tr>
  <tr><td>sitkVectorFloat32</td><td>Multi-component of 32 bit float</td></tr>
  <tr><td>sitkVectorFloat64</td><td>Multi-component of 64 bit float</td></tr>
  <tr><td>sitkLabelUInt8</td><td>RLE label of unsigned 8 bit integers</td></tr>
  <tr><td>sitkLabelUInt16</td><td>RLE label of unsigned 16 bit integers</td></tr>
  <tr><td>sitkLabelUInt32</td><td>RLE label of unsigned 32 bit integers</td></tr>
  <tr><td>sitkLabelUInt64</td><td>RLE label of unsigned 64 bit integers</td></tr>
</table>

There is also `sitkUnknown`, which is used for undefined or erroneous pixel ID's. It has a value of -1.

The 64-bit integer types are not available on all distributions. When not available the value is `sitkUnknown`.

In [ ]:
print(image.GetPixelIDTypeAsString())
print(image.GetPixelID())
print(sitk.sitkUInt8)

## Image Construction

There are a variety of ways to create an image. 

The following components are required for a complete definition of an image:
<ol>
<li>Pixel type [fixed on creation, no default]: unsigned 32 bit integer, sitkVectorUInt8, etc., see list above.</li>
<li> Sizes [fixed on creation, no default]: number of pixels/voxels in each dimension. This quantity implicitly defines the image dimension.</li>
<li> Origin [default is zero]: coordinates of the pixel/voxel with index (0,0,0) in physical units (i.e. mm).</li>
<li> Spacing [default is one]: Distance between adjacent pixels/voxels in each dimension given in physical units.</li>
<li> Direction matrix [default is identity]: mapping, rotation, between direction of the pixel/voxel axes and physical directions.</li>
</ol>

Initial pixel/voxel values are set to zero.

In [ ]:
image_3D = sitk.Image(256, 128, 64, sitk.sitkInt16)
image_2D = sitk.Image(64, 64, sitk.sitkFloat32)
image_2D = sitk.Image([32,32], sitk.sitkUInt32)
image_RGB = sitk.Image([128,64], sitk.sitkVectorUInt8, 3)

### Basic Image Attributes

You can change the image origin, spacing and direction. Making such changes to an image already containing data should be done cautiously. 

In [ ]:
image_3D.SetOrigin((78.0, 76.0, 77.0))
image_3D.SetSpacing([0.5,0.5,3.0])

print(image_3D.GetOrigin())
print(image_3D.GetSize())
print(image_3D.GetSpacing())
print(image_3D.GetDirection())

Image dimension queries:

In [ ]:
print(image_3D.GetDimension())
print(image_3D.GetWidth())
print(image_3D.GetHeight())
print(image_3D.GetDepth())

What is the depth of a 2D image?

In [ ]:
print(image_2D.GetSize())
print(image_2D.GetDepth())

Pixel/voxel type queries: 

In [ ]:
print(image_3D.GetPixelIDValue())
print(image_3D.GetPixelIDTypeAsString())
print(image_3D.GetNumberOfComponentsPerPixel())

What is the dimension and size of a Vector image and its data?

In [ ]:
print(image_RGB.GetDimension())
print(image_RGB.GetSize())
print(image_RGB.GetNumberOfComponentsPerPixel())

### Accessing Pixels and Slicing

The Image class's member functions `GetPixel` and `SetPixel` provide an interface for pixel access.

In [ ]:
help(image_3D.GetPixel)

In [ ]:
print(image_3D.GetPixel(0, 0, 0))
image_3D.SetPixel(0, 0, 0, 1)
print(image_3D.GetPixel(0, 0, 0))

# This can also be done using pythonic notation.
print(image_3D[0,0,1])
image_3D[0,0,1] = 2
print(image_3D[0,0,1])

Slicing of SimpleITK images returns a copy of the image data, but only a subset of pixels.

The syntax is the same as NumPy arrays, which is very similar to MATLAB.

In [ ]:
# Plot the original image
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
ax.imshow(sitk.GetArrayFromImage(image), cmap='gray')
ax.set_title('Original')

# Brute force sub-sample
image_subsampled = image[::16,::16]
ax = fig.add_subplot(1, 2, 2)
ax.imshow(sitk.GetArrayFromImage(image_subsampled), cmap='gray')
ax.set_title('Subsampled')

We can take a 2D slice of a 3D image for visualization.

In [ ]:
# Data from http://loci.wisc.edu/software/sample-data
cells = sitk.ReadImage('Data/NESb_C2_TP1.tiff')
print('Size', cells.GetSize())

cells_slice = cells[:,:,4]
print('Slice size', cells_slice.GetSize())

plt.imshow(sitk.GetArrayFromImage(cells_slice), cmap='gray')

## Exercises

### Exercise 1: Image addition and physical space

ITK does not allow pixel-based addition of images when they do not occupy the same location in physical space.

Uncomment the addition operation below, and observe the error that occurs.

Comment the `SetOrigin`, `SetSpacing`, and `SetDirection` lines.  Does the error go away? Why? Can any of these attributes be modified and addition still occur?

In [ ]:
image_a = sitk.Image(24, 24, sitk.sitkUInt8)
image_a[0,0] = 2

image_b = sitk.Image(image_a.GetSize(), sitk.sitkUInt8)
image_b.SetDirection([0, 1, 0.5, 0.5])
image_b.SetSpacing([0.5, 0.8])
image_b.SetOrigin([10.0, 20.0])
image_b[0,0] = 3

#image_c = sitk.Add(image_a, image_b)
#print(image_c[0,0])

### Exercise 2: Image physical extent

In the figure below, two images are plotted at their physical location in space.

By changing the *Origin* and *Spacing* of the image *b* only, make them occupy the same area in physical space.

*Hint*: The length of an image with spacing, $\Delta x$ and size $n$ is $n \Delta x$. For two lengths to be the same, $\Delta x_1 n_1 = \Delta x_2 n_2$.

In [ ]:
def show_in_physical_space(image, column):
    fig = plt.figure()
    ax = fig.add_subplot(2, 2, column)
    first_index = (0, 0)
    left, bottom = image.TransformIndexToPhysicalPoint(first_index)
    size = image.GetSize()
    # Since indexes count from zero, we must subtract 1
    last_index = (size[0] - 1, size[1] - 1)
    right, top = image.TransformIndexToPhysicalPoint(last_index)
    extent = (left, right, bottom, top)
    print('extent', extent)
    ax.imshow(sitk.GetArrayFromImage(image), extent=extent)
    ax.set_xlim(0.0, 100.0)
    ax.set_ylim(0.0, 100.0)
    plt.show()

In [ ]:
image_a = sitk.Image(20, 20, sitk.sitkUInt8)
image_a.SetOrigin([10.0, 20.0])
image_a.SetSpacing([4.0, 2.0])

show_in_physical_space(image_a, 1)

image_b = sitk.Image(10, 10, sitk.sitkUInt8)
image_b.SetOrigin([5.0, 5.0])
image_b.SetSpacing([2.0, 1.0])

show_in_physical_space(image_b, 2)